In [ ]:
#pip list |grep tensorflow

In [ ]:
!pip install efficientnet
#!pip install tf-nightly
#pip install tensorflow-addons

In [ ]:
# Cloud Storage
from google.cloud import storage
storage_client = storage.Client(project='credible-art-300513')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from kaggle_datasets import KaggleDatasets
import random

GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH)

In [ ]:
import re, math
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import glob
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import tensorflow_addons as tfa
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from tensorflow.keras import mixed_precision
import efficientnet.tfkeras as efn
import gc
from tensorflow.keras.layers.experimental import preprocessing
import albumentations as albu
import pandas_profiling as pp
from collections import Counter
import seaborn as sns
from matplotlib import cm
print('TensorFlow version =',tf.__version__)

In [ ]:
#gcpbuc = storage_client.create_bucket("abhijeet0706-bucket") #Bucket Already Exists

In [ ]:
bucket = storage_client.get_bucket("abhijeet0706-bucket")

In [ ]:
#gpulist = tf.config.list_physical_devices('GPU')

In [ ]:
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

In [ ]:
#tf.test.gpu_device_name()

In [ ]:
#help(tf.keras.applications)

In [ ]:
#Not to be used with TPU
#Reduce memory requirements during training layers
#mixed_precision.experimental.set_policy('mixed_float16')
#mixed_precision.experimental.global_policy()

In [ ]:
"""
if len(gpulist)==1: 
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
else: strategy = tf.distribute.MirroredStrategy()
print(strategy)
"""

In [ ]:

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)


In [ ]:
IMAGE_SIZE = [512,512,3]
dim = IMAGE_SIZE[0]
EPOCHS = 20
print(dim)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

TRAINING_FILENAMES1 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192' + '/train/*.tfrec')
VALIDATION_FILENAMES1 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192' + '/val/*.tfrec')
TEST_FILENAMES1 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192' + '/test/*.tfrec')

TRAINING_FILENAMES2 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-224x224' + '/train/*.tfrec')
VALIDATION_FILENAMES2 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-224x224' + '/val/*.tfrec')
TEST_FILENAMES2 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-224x224' + '/test/*.tfrec')

TRAINING_FILENAMES3 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-331x331' + '/train/*.tfrec')
VALIDATION_FILENAMES3 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-331x331' + '/val/*.tfrec')
TEST_FILENAMES3 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-331x331' + '/test/*.tfrec')

TRAINING_FILENAMES4 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-512x512' + '/train/*.tfrec')
VALIDATION_FILENAMES4 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-512x512' + '/val/*.tfrec')
TEST_FILENAMES4 = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-512x512' + '/test/*.tfrec')

In [ ]:
TRAINING_FILE = []
VALIDATION_FILE = []
TEST_FILENAMES = []
TRAINING_FILE.extend(TRAINING_FILENAMES1)
TRAINING_FILE.extend(TRAINING_FILENAMES2)
TRAINING_FILE.extend(TRAINING_FILENAMES3)
TRAINING_FILE.extend(TRAINING_FILENAMES4)
VALIDATION_FILE.extend(VALIDATION_FILENAMES1)
VALIDATION_FILE.extend(VALIDATION_FILENAMES2)
VALIDATION_FILE.extend(VALIDATION_FILENAMES3)
VALIDATION_FILE.extend(VALIDATION_FILENAMES4)
TEST_FILENAMES.extend(TEST_FILENAMES1)
#TEST_FILENAMES.extend(TEST_FILENAMES2)
#TEST_FILENAMES.extend(TEST_FILENAMES3)
#TEST_FILENAMES.extend(TEST_FILENAMES4)

In [ ]:
#TRAINING_FILE

In [ ]:
random.shuffle(TRAINING_FILE)
random.Random(4).shuffle(VALIDATION_FILE)

In [ ]:
"""
train_files2=[]
val_files2=[]
for i in range(len(TRAINING_FILE)//3):
    train_files2.append(TRAINING_FILE[i])
for j in range(len(VALIDATION_FILE)//3):
    val_files2.append(VALIDATION_FILE[j])
TRAINING_FILENAMES = train_files2
VALIDATION_FILENAMES = val_files2
"""

In [ ]:
TRAINING_FILENAMES = TRAINING_FILE
VALIDATION_FILENAMES = VALIDATION_FILE

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']

In [ ]:
def random_blockout(img, sl=0.1, sh=0.08, rl=0.4):
    tf.random.set_seed(5)
    p=random.random()
    if p>=0.00:
        w, h, c = IMAGE_SIZE[0], IMAGE_SIZE[1], 3
        origin_area = tf.cast(h*w, tf.float32)

        e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
        e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

        e_height_h = tf.minimum(e_size_h, h)
        e_width_h = tf.minimum(e_size_h, w)
        
        e_height_h = tf.random.uniform(shape=[], minval=1, maxval=e_height_h, dtype=tf.int32, seed=10)
        e_width_h = tf.random.uniform(shape=[], minval=1, maxval=e_width_h, dtype=tf.int32, seed=10)
        
        #erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32, seed=10)
        #erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32, seed=10)
        erase_height = tf.random.uniform(shape=[], minval=0, maxval=e_height_h, dtype=tf.int32, seed=10)
        erase_width = tf.random.uniform(shape=[], minval=0, maxval=e_width_h, dtype=tf.int32, seed=10)
        
        #Erase area is black
        #erase_area = tf.zeros(shape=[erase_height, erase_width, c])
        #Erase area is opaque-black
        erase_area = tf.random.uniform(shape=[erase_height, erase_width, c], minval=0, maxval=.3, dtype=tf.float32, seed=10)
        #erase_area = tf.random.truncated_normal(shape=[erase_height, erase_width, c], mean=0.5, stddev=0.24, dtype=tf.float32, seed=10)
        #erase_area = tf.cast(erase_area, tf.uint8)

        pad_h = h - erase_height
        pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32, seed=10)
        pad_bottom = pad_h - pad_top

        pad_w = w - erase_width
        pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32, seed=10)
        pad_right = pad_w - pad_left

        erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
        erase_mask = tf.squeeze(erase_mask, axis=0)
        erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

        return tf.cast(erased_img, img.dtype)
    else:
        return tf.cast(img, img.dtype)



In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, [dim,dim], method=tf.image.ResizeMethod.BICUBIC, antialias=True)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, IMAGE_SIZE) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    # for free on TPU. Data pipeline code is executed on the "CPU"
    # part of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = random_blockout(image)
    #if label%2==0:
    #image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    #image = tf.image.random_saturation(image, 0.8, 1.2)
    #image = tf.image.random_hue(image, 0.2)
    #image = tf.image.random_brightness(image, 0.1)
    #image = tfa.image.rotate(image, 30* math.pi / 180, interpolation='nearest')
    #image = tf.image.rgb_to_grayscale(image)
    #image = tf.image.random_crop(image,size=(dim,dim,3))
    #image = tf.image.random_jpeg_quality(image,75,95)
    #image = tf.image.resize(image, [dim,dim], method=tf.image.ResizeMethod.BICUBIC, antialias=True)
    return image, label

In [ ]:
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    #Augmenting only half epochs and adding original and augmented one-by-one
    #epochhalf = EPOCHS//2
    dataset = dataset.repeat(EPOCHS)
    #dataset = dataset.shuffle(2048,reshuffle_each_iteration=True)
    #dataset = dataset.shuffle(2048)
    #print(type(dataset))
    """
    datacon = dataset.map(data_augment, num_parallel_calls=AUTO)
    
    for i in range(epochhalf):
        # Augmenting one dataset
        dataaug = dataset.map(data_augment, num_parallel_calls=AUTO)
        # Adding above augmented one dataset
        datacon = datacon.concatenate(dataaug)
        # Adding normal dataset
        if ((2*(i+1))+1)<=EPOCHS:
            datacon = datacon.concatenate(dataset)
    
    for i in range(EPOCHS-1):
        # Augmenting one dataset
        dataaug = dataset.map(data_augment, num_parallel_calls=AUTO)
        # Adding above augmented one dataset
        datacon = datacon.concatenate(dataaug)
    
    dataset = datacon
    """
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    #print(type(dataset))
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [ ]:
def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec
    # files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:
# Define the batch size. This will be 16 with TPU off and 128 (=16*8) with TPU on
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
#BATCH_SIZE = 32

ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

In [ ]:
#dir()
#locals()
#globals()

In [ ]:
#del KaggleDatasets
#_ = gc.collect()

In [ ]:
#print(len(list(ds_valid.unbatch().as_numpy_iterator())))

In [ ]:
np.set_printoptions(threshold=15, linewidth=80)

print("Training data shapes:")
for image, label in ds_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

In [ ]:
print("Validation data shapes:")
for image, idnum in ds_valid.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Validation data IDs:", idnum.numpy().astype('U')) # U=unicode string

In [ ]:
print("Test data shapes:")
for image, idnum in ds_test.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

In [ ]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case,
                                     # these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is
    # the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square
    # or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_ylim(0.0,1.15)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:

ds_iter = iter(ds_train)
one_batch = next(ds_iter)
display_batch_of_images(one_batch)


In [ ]:
weight_class = True
if weight_class:
    raw_training_dataset = load_dataset(TRAINING_FILENAMES, labeled = True, ordered = False)

    label_counter = Counter()
    for images, labels in raw_training_dataset:
        label_counter.update([labels.numpy()])

    del raw_training_dataset

In [ ]:
if weight_class:
#    TARGET_NUM_PER_CLASS = 300
    TARGET_LOG = 8.5 #Equilalent to count of 362

    def get_weight_for_class(class_id):
        counting = label_counter[class_id]
        logcount = np.log2(counting)
        if logcount > TARGET_LOG:
            weight = round(1.0 - ((logcount-TARGET_LOG)/5),2)
        else:
            weight = round(1.0 + ((TARGET_LOG-logcount)/4),2)
        return weight

    weight_per_class = {class_id: get_weight_for_class(class_id) for class_id in range(104)}

    print("Label Counts: ",label_counter)
    print("Weight Per Class: ",weight_per_class)

In [ ]:
if weight_class:
    data = pd.DataFrame.from_dict(weight_per_class, orient='index', columns=['class_weight'])
    plt.figure(figsize=(30, 8))

    #barplot color based on value
    bplot = sns.barplot(x=data.index, y='class_weight', data=data, palette= cm.Blues(data['class_weight']*0.22));
    for p in bplot.patches:
        bplot.annotate(format(p.get_height(), '.1f'), 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'center', 
                       xytext = (0, 9), 
                       textcoords = 'offset points')
    plt.xlabel("Class", size=14)
    plt.ylabel("Class weight (inverse of %)", size=14)

In [ ]:
"""
Try various activations
Try various dropouts
Try L1 and L2 kernel regularization
Try varoius transfer learning
Try different Kernel initializers (Random-normal, random-uniform, glorot-normal, glorot-uniform, ones, zeroes) - Not required
Try lower batch size
Try different kernel weight constraints
Try different optimizers
Try different augmentations
Try different image sizes
Try different momentum
Try different learning rates
Try NASNetLarge
Try EfficientNetB4, B6
Try 'noisy-student' weights instead of 'imagenet' - Only valid for EfficientNet
Try including top layer and modify last few layers - Only prediction layer after GAP in all models
Try training last layer of base model
Try ensemble and Top1,Top3,Top5 values
Try lower LR (<=0.0001) for base model layers and higher (>=0.0005) for top FC layers


Notes -
tf.keras.applications.EfficientNetB0() -
Use "import efficientnet.tfkeras.EfficientNetB4()" for this. Or B5, B6.... 

tf.keras.applications.NASNetLarge() -
Loading `imagenet` weights, `input_shape` should be (331, 331, 3)

tf.keras.applications.NASNetMobile() -
Loading `imagenet` weights, `input_shape` should be (224, 224, 3)

tf.keras.applications.MobileNetV3Large() -
Takes a lot of epochs > 10 to start finding validation accuracy (tried LR from 0.0003, 0.0005, 0.001)
"""

In [ ]:
def create_base_MobileNetV2():    
    pretrained_model = tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False,input_shape=(dim,dim,3))
#    pretrained_model.trainable = False
    """
    # To set LayerX and subsequent layers in base pretrained model as trainable
    pretrained_model.trainable = True
    set_trainable = False
    for layer in pretrained_model.layers:
        if layer.name == 'conv3_block1_1_conv':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    """
    return pretrained_model

def create_base_MobileNetV3Large():    
    pretrained_model = tf.keras.applications.MobileNetV3Large(weights='imagenet',include_top=False,input_shape=(dim,dim,3))
#    pretrained_model.trainable = False
    return pretrained_model

def create_base_DenseNet121():    
    pretrained_model = tf.keras.applications.DenseNet121(weights='imagenet',include_top=False,input_shape=(dim,dim,3))
#    pretrained_model.trainable = False    
    return pretrained_model

def create_base_DenseNet201():    
    pretrained_model = tf.keras.applications.DenseNet201(weights='imagenet',include_top=False,input_shape=(dim,dim,3))
#    pretrained_model.trainable = False    
    return pretrained_model

def create_base_Xception():    
    pretrained_model = tf.keras.applications.Xception(weights='imagenet',include_top=False,input_shape=(dim,dim,3))
#    pretrained_model.trainable = False
    return pretrained_model

def create_base_InceptionResNet():    
    pretrained_model = tf.keras.applications.InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(dim,dim,3))
#    pretrained_model.trainable = False    
    return pretrained_model

def create_base_EfficientNetB4():    
    pretrained_model = efn.EfficientNetB4(weights='noisy-student',include_top=False, input_shape=(dim,dim,3))
#    pretrained_model.trainable = False
    return pretrained_model

def create_base_EfficientNetB6():    
    pretrained_model = efn.EfficientNetB6(weights='noisy-student',include_top=False, input_shape=(dim,dim,3))
#    pretrained_model.trainable = False
    return pretrained_model

In [ ]:
def add_Model(base_model,opt,learn_r,moment,activ,w_constr):
    model = tf.keras.Sequential([
    # To a base pretrained on ImageNet to extract features from images...
    base_model,
    # ... attach a new head to act as a classifier.
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512,activation=activ,kernel_initializer='glorot_uniform',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                                  kernel_constraint=maxnorm(w_constr)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(CLASSES), dtype='float32', activation='softmax')
    ])
        
    if opt=='SGD':
        optim = tf.keras.optimizers.SGD(learning_rate=learn_r,momentum=moment)
    elif opt=='RMSprop':
        optim = tf.keras.optimizers.RMSprop(learning_rate=learn_r,momentum=moment)
    elif opt=='Adam':
        optim = tf.keras.optimizers.Adam(learning_rate=learn_r)
    elif opt=='Adamax':
        optim = tf.keras.optimizers.Adamax(learning_rate=learn_r)
    elif opt=='Nadam':
        optim = tf.keras.optimizers.Nadam(learning_rate=learn_r)
        
    model.compile(optimizer=optim, loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    return model

In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

In [ ]:
#optim1 = ['SGD','RMSprop','Adam', 'Adamax', 'Nadam']
#learn_lr = [0.0001,0.001,0.01,0.1]
#momentum1 = [0.0,0.3,0.5,0.8]
momentum1 = 0.0
#activ1 = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear','elu','gelu','selu']
w_constr1= 10
optim1 = "Adam"

In [ ]:
"""
with strategy.scope():
#    base = create_base_MobileNetV2()
#    base = create_base_MobileNetV3Large()
#    base = create_base_DenseNet121()
#    base = create_base_DenseNet201()
    base = create_base_Xception()
#    base = create_base_InceptionResNet()
#    base = create_base_EfficientNetB4()
#    base = create_base_EfficientNetB6()
    model = add_Model(base,optim1,0.0003,momentum1,'elu',w_constr1)

#print(i," Training Accuracy = ",model1.history['sparse_categorical_accuracy'][16:20])
#print(i," Validation Accuracy = ",model1.history['val_sparse_categorical_accuracy'][16:20])
"""

In [ ]:
#p_model = efn.EfficientNetB4(weights=None,include_top=False, input_shape=(dim,dim,3))
#plot_model(p_model, to_file='pretrained_model.png', show_shapes=True)
#Image(filename='pretrained_model.png')

In [ ]:
"""
Top Layers
VGG16 - 7,7,512 Flatten,2d-4096,1d
VGG19 - 7,7,512 Flatten,2d-4096,1d
ResNet50 23M - 7,7,2048 GAP,1d
DenseNet121 7M - 16,16,1024 GAP,1d
DenseNet201
InceptionV3 23M - 8,8,2048 GAP,1d
InceptionResNetV2 55M - 5,5,1536 GAP,1d
MobileNetV2 3M - 7,7,1280 GAP,1d
NASNetMobile 5M - 7,7,1056 GAP,1d
NASNetLarge - 7,7,4032 GAP,1d
Xception 22M - 7,7,2048 GAP,1d
EfficientNetB4 19M - 7,7,1792 GAP,1d-1792,1d
EfficientNetB6
MobileNetV3Large 5M
"""

In [ ]:
#model.summary()

In [ ]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
chkpt_callback = tf.keras.callbacks.ModelCheckpoint('AugXception-512ww.h5', monitor='val_loss', mode='auto', save_best_only=True,
                                                    save_weights_only=True,verbose=1)

In [ ]:
"""
model1 = model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
                   callbacks=[reduce_lr,chkpt_callback]
#                   callbacks=[reduce_lr]
                   ,class_weight = weight_per_class
                   ,verbose=1)
"""

In [ ]:
""" Kaggle-Petals-Metal
: Validation Accuracy% imagenet=I noisy-student=NS
GPU: (192,192) I-VGG16 Top_train epoch20 LRS: 75mins 50%
GPU: (192,192) I-ResNet50 Top_train 1d epoch20 LRS: 75mins 14%
TPU: (192,192) I-ResNet50 Top_train 1d epoch20 LRS: 5mins 14%
TPU: (192,192) I-VGG19 Top_train 1d epoch20 LRS: 5mins 40%
TPU: (192,192) I-VGG16 Top_train 1d epoch20 LRS: 5mins 42%
TPU: (192,192) I-VGG16 Top_train 2d epoch20 RLRP: 5mins 70%
TPU: (192,192) I-ResNet50 Top_train 2d epoch20 RLRP: 10mins 26%
TPU: (192,192) I-DenseNet121 Top_train 2d epoch20 RLRP: 10mins 82% (Train 98%)
TPU: (224,224) I-VGG16 Top_train 2d epoch20 RLRP: 7mins 72% (Train 84%)
TPU: (224,224) I-DenseNet121 Top_train 2d epoch20 RLRP: 8mins 83.5% (Train 98.5%)
TPU: (224,224) I-InceptionV3 Top_train 2d epoch20 RLRP: 8mins 77% (Train 98%)
TPU: (224,224) I-MobileNetV2 Top_train 2d epoch20 RLRP: 6mins 81% (Train 99.4%)
TPU: (224,224) I-NASNetMobile Top_train 2d epoch20 RLRP: 6mins 75% (Train 95.5%)
TPU: (224,224) I-InceptionResNetV2 Top_train 2d epoch20 RLRP: 6mins 81% (Train 96.5%)
TPU: (224,224) I-Xception Top_train 2d epoch20 RLRP: 7mins 80% (Train 99%)
TPU: (224,224) Aug I-DenseNet121 Top_train 2d epoch20 RLRP: 10mins 76% (Train 93%)
TPU: (224,224) I-NASNetLarge Top_train 2d epoch20 RLRP l2-0.005: 30mins 74% (Train 73%)
TPU: (224,224) I-NASNetLarge Top_train 2d epoch20 RLRP l2-0.001: 30mins 74% (Train 73%)
TPU: (224,224) I-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.001 0.5d: 10mins 81.7% (Train 86%)
TPU: (224,224) I-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.0001 0.5d: 10mins 82.8% (Train 90%)
TPU: (224,224) 128b Aug I-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.0001 0.5d: 15mins 81% (Train 86%)
TPU: (224,224) 192b Aug I-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.0001 0.5d: 13mins 80% (Train 86%)
TPU: (224,224) 256b Aug I-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.0001 0.5d: 8mins 81% (Train 88%)
TPU: (224,224) 256b Aug I-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.0001 0.2d: 8mins 82% (Train 93%)
TPU: (224,224) 256b Aug NS-EfficientNetB4 Top_train 2d epoch20 RLRP l2-0.0001 0.2d: 8mins 82% (Train 88%)
TPU: (224,224) 256b Aug I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu: 15mins 92% (Train 100%)
TPU: (224,224) 256b Aug I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu: 18mins 92.4% (Train 100%)
TPU: (224,224) 256b Aug I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 18mins 92.5% (Train 100%)
TPU: (224,224) 256b Aug I-DenseNet121 20%_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 14mins 89% (Train 98%)
TPU: (224,224) 256b Aug I-DenseNet121 20%_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 14mins 89.5% (Train 99%)
TPU: (224,224) 256b Aug I-DenseNet121 40%_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 14mins 89.5% (Train 99%)
TPU: (224,224) 256b Aug I-DenseNet121 40%_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 14mins 92% (Train 100%)
TPU: (224,224) 256b Aug I-DenseNet121 60%_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 14mins 92.2% (Train 100%)
TPU: (224,224) 256b Aug I-ResNet50 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.001: 14mins 88% (Train 100%)
TPU: (224,224) 256b I-ResNet50 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.001: 14mins 89% (Train 100%)
TPU: (224,224) 256b I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 14mins 92.6% (Train 100%)
TPU: (224,224) 256b I-InceptionV3 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 14mins 90% (Train 100%)
TPU: (224,224) 256b I-InceptionResNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 25mins 91% (Train 100%)
TPU: (224,224) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 13mins 90% (Train 100%)
TPU: (224,224) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.001: 25mins 91% (Train 100%)
TPU: (224,224) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 13mins 91.4% (Train 100%)
TPU: (224,224) 256b I-NASNetMobile All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 60mins 90% (Train 100%)
TPU: (224,224) 256b I-Xception All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 12mins 91.7% (Train 100%)
TPU: (224,224) 256b NS-EfficientNetB4 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 25mins 94.4% (Train 100%)
TPU: (224,224) 256b I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 18mins 93.2% (Train 100%)
TPU: (224,224) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 13mins 86% (Train 99%)
TPU: (224,224) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0001: 13mins 91% (Train 100%) *40E
TPU: (224,224) 256b I-Xception All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0003: 12mins 92.2% (Train 100%)
TPU: (224,224) 256b I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0003: 12mins 92.6% (Train 100%)
TPU: (512,512) 256b AugI-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu: 45mins 93.5% (Train 100%)
TPU: (512,512) 256b I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 50mins 94.4% (Train 100%)
TPU: (512,512) 256b I-Xception All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0003: 35mins 93.4% (Train 100%)
TPU: (512,512) 256b NS-EfficientNetB4 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 70mins 96.2% (Train 100%)
TPU: (512,512) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0005: 25mins 93% (Train 100%)
TPU: (512,512) 256b I-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 25mins 93.1% (Train 100%) *40E
TPU: (512,512) 256b I-DenseNet121 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0002: 50mins 94.5% (Train 100%) *40E
TPU: (512,512) 256b I-Xception All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0002: 35mins 94.3% (Train 100%) *40E
TPU: (512,512) 256b NS-EfficientNetB4 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0002: 70mins 96.4% (Train 100%)
TPU: (512,512) 256b I-DenseNet201 All_train 2d epoch20 RLRP l2-0.0001 0.2d Nadam-Relu lr-0.0002: 70mins 94.8% (Train 100%) *40E
TPU: (512,512) 256b I-DenseNet201 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0002: 70mins 95.0% (Train 100%) *40E
TPU: (512,512) 256b AugI-DenseNet201 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 80mins 95.0% (Train 100%) *40E
TPU: (512,512) 256b AugI-NS-EfficientNetB6 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 100mins 96.1% (Train 100%)
TPU: (512,512) 256b AugI-Xception All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0003: 35mins 94.5% (Train 100%)
TPU: (512,512) 256b AugI-MobileNetV2 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 30mins 93.5% (Train 100%)
TPU: (512,512) 256b AugI-MobileNetV3L All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0005: 30mins 94.0% (Train 100%) *80E
TPU: (512,512) 256b I-MobileNetV3L All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.001: 30mins 94.0% (Train 100%) *80E
TPU: (512,512) 256b I-DenseNet201 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 80mins 95.2% (Train 100%) *40E
TPU: (512,512) 256b I-NS-EfficientNetB6 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003: 100mins 96% (Train 100%) *40E
TPU: (512,512) 256b AugI-Xception All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-elu lr-0.0003 wght: 40mins 94.4% (Train 100%) *40E
TPU: (512,512) 256b AugI-MobileNetV3L All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.001 wght: 25mins 94.1% (Train 100%) *80E
TPU: (512,512) 256b AugI-NS-EfficientNetB6 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003 wght: 100mins 96.3% (Train 100%) *40E
TPU: (512,512) 256b AugI-DenseNet201 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003 wght: 80mins 95% (Train 100%) *40E
TPU: (672,672) 256b AugI-NS-EfficientNetB6 All_train 2d epoch20 RLRP l2-0.0001 0.2d Adam-Relu lr-0.0003 wght: 100mins 95.5% (Train 100%) *40E

TPU Kaggle: 17k (224,224) images DenseNet121 Top_train 2d epoch20 RLRP:
optimizer = ['SGD' 68%, 'RMSprop' 82%, 'Adagrad' 72%, 'Adadelta' 10%, 'Adam' 82%, 'Adamax' 81%, 'Nadam' 82%] 25mins
FilteredOut - Adadelta, Adagrad

SGD lr0.1 mmntm0.8 82%
RMSProp lr0.001 mmntm0.8 82.7%
Adam lr0.001 82.95%
Adamax lr0.01 82.8%
Nadam lr0.001 82.5%

relu 82.9%
elu 82.4%

Dropouts-
TPU Kaggle: 51k (224,224) 256b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.4: 7mins 84.5%
TPU Kaggle: 51k (224,224) 256b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.5: 7mins 84.8%
TPU Kaggle: 51k (224,224) 256b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.6: 7mins 84.6%

kernel weight constraint = 4
TPU Kaggle: 51k (224,224) 128b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.5: 12mins 84.6%
TPU Kaggle: 51k (224,224) 64b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.5: 18mins 84%

Regularization-
TPU Kaggle: 51k (224,224) 128b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.5:
'l1' 0.01: 12mins 67%
'l2' 0.01: 12mins 79%
'l1_l2' 0.01: 12mins 67%

TPU Kaggle: 51k (224,224) 256b images DenseNet121 Top_train 2d epoch20 RLRP 'Adam',0.001,'relu',0.5:
'l2' 0.01: 8mins 80%
kernel weight constraint = 5
'l2' 0.01: 8mins 80.5%
'l2' 0.005 dropout0.5: 8mins 83.5%
'l2' 0.005 dropout0.3: 8mins 83.4%
"""

In [ ]:
!ls

In [ ]:
"""
# Learning Rate Schedule for Fine Tuning #
def exponential_lr(epoch,start_lr = 0.00001, min_lr = 0.00001, max_lr = 0.00005,
                   rampup_epochs = 5, sustain_epochs = 0, exp_decay = 0.8):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) / rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) * exp_decay**(epoch - rampup_epochs - sustain_epochs) + min_lr)
        return lr
    return lr(epoch,start_lr,min_lr,max_lr,rampup_epochs,sustain_epochs,exp_decay)

lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [exponential_lr(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))
"""

In [ ]:
#model.save_weights('EfficientNetB4-512.h5')

In [ ]:
# List GCP files
blobs = storage_client.list_blobs("abhijeet0706-bucket")
for blob in blobs:
    if blob.name.startswith(('Kaggle')):
        print(blob.name)

In [ ]:
"""
# Upload Kaggle file
#kaggleblob = '/kaggle/input/tpu-getting-started/sample_submission.csv'
kaggleblob = '/kaggle/working/AugXception-512ww.h5'
gcpblob = 'Kaggle-Petals-Metal/AugXception-512ww.h5'
blob = bucket.blob(gcpblob)
blob.upload_from_filename(kaggleblob)
"""

In [ ]:
# Save the entire model as a SavedModel.
#!mkdir -p saved_model
#model.save('saved_model/VGG16_2train')

In [ ]:
"""
display_training_curves(
    model1.history['loss'],
    model1.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    model1.history['sparse_categorical_accuracy'],
    model1.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)
"""

In [ ]:
def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
# View Kaggle Working folder files
os.listdir('/kaggle/working/')

In [ ]:

# Download GCP files
kagglefolder = '/kaggle/working/'
os.makedirs(kagglefolder, exist_ok = True) #If folder already exists, nothing will happen
blobs = storage_client.list_blobs("abhijeet0706-bucket")
for blob in blobs:
    if blob.name.startswith(('Kaggle')):
        kagglefile = str(blob.name).split('/')[1]
        print(kagglefile)
        kagglepath = os.path.join(kagglefolder,kagglefile)
        blob.download_to_filename(kagglepath)


In [ ]:
# Delete files from kaggle output folder
#!rm -rf  /kaggle/working/sample_submission.csv

In [ ]:
# Best Model here -
with strategy.scope():    
    base2 = create_base_EfficientNetB6()
    model2 = add_Model(base2,'Adam',0.0003,momentum1,'relu',w_constr1)

model2.load_weights('./AugEfficientNetB6-512ww.h5')

In [ ]:

# 2nd Best Model here -
with strategy.scope():    
    base3 = create_base_DenseNet201()
    model3 = add_Model(base3,'Adam',0.0003,momentum1,'relu',w_constr1)

model3.load_weights('./AugDenseNet201-512ww.h5')


In [ ]:

# 3rd Best Model here -
with strategy.scope():    
    base4 = create_base_Xception()
    model4 = add_Model(base4,'Adam',0.0003,momentum1,'elu',w_constr1)

model4.load_weights('./AugXception-512ww.h5')


In [ ]:

# 4th Best Model here -
with strategy.scope():    
    base5 = create_base_MobileNetV3Large()
    model5 = add_Model(base5,'Adam',0.0001,momentum1,'relu',w_constr1)

model5.load_weights('./AugMobileNetV3L-512ww.h5')


In [ ]:
"""
# 5th Best Model here -
with strategy.scope():    
    base6 = create_base_InceptionResNet()
    model6 = add_Model(base6,'Adam',0.0003,momentum1,'relu',w_constr1)

model6.load_weights('./InceptionResNetV2-512.h5')
"""

In [ ]:
cmdataset = get_validation_dataset(ordered=True)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()
cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()
labels = range(len(CLASSES))

In [ ]:
"""
list1 = list(labels_ds.as_numpy_iterator())
df1 = pd.DataFrame(list1, columns=['Actual'])
df1.head(3)
"""

In [ ]:
"""
list2 = []
for i in cm_probabilities:
    list2.append(i.tolist())

df2 = pd.DataFrame(list2, columns=CLASSES)
df2.head(3)
"""

In [ ]:
"""
df1.to_csv('df1.csv')
df2.to_csv('df2.csv')
"""

In [ ]:
cm_probabilities = model2.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)
cmat = confusion_matrix(cm_correct_labels,cm_predictions,labels=labels)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize
score = f1_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
precision = precision_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
recall = recall_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
display_confusion_matrix(cmat, score, precision, recall)

In [ ]:
"""
cm_probabilities = model3.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)
cmat = confusion_matrix(cm_correct_labels,cm_predictions,labels=labels)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize
score = f1_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
precision = precision_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
recall = recall_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
display_confusion_matrix(cmat, score, precision, recall)
"""

In [ ]:
"""
cm_probabilities = model4.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)
cmat = confusion_matrix(cm_correct_labels,cm_predictions,labels=labels)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize
score = f1_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
precision = precision_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
recall = recall_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
display_confusion_matrix(cmat, score, precision, recall)
"""

In [ ]:
"""
cm_probabilities = model5.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)
cmat = confusion_matrix(cm_correct_labels,cm_predictions,labels=labels)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize
score = f1_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
precision = precision_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
recall = recall_score(cm_correct_labels,cm_predictions,labels=labels,average='macro')
display_confusion_matrix(cmat, score, precision, recall)
"""

In [ ]:
dataset = get_validation_dataset()
dataset = dataset.unbatch().batch(25)
batch = iter(dataset)

In [ ]:
images, labels = next(batch)
probabilities = model2.predict(images)
predictions = np.argmax(probabilities, axis=-1)
display_batch_of_images((images, labels), predictions)

In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)

# Get image ids from test set and convert to unicode
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

In [ ]:
probabilities2 = model2.predict(test_images_ds)
predictions2 = np.argmax(probabilities2, axis=-1)
maxprob2 = np.amax(probabilities2, axis=-1)
print(predictions2)
print(maxprob2)

In [ ]:
np.savetxt('submission1.csv',np.rec.fromarrays([test_ids,predictions2,maxprob2]),fmt=['%s','%d','%.7f'],delimiter=',',
           header='id,label,prob',comments='')

In [ ]:
probabilities3 = model3.predict(test_images_ds)
predictions3 = np.argmax(probabilities3, axis=-1)
maxprob3 = np.amax(probabilities3, axis=-1)
print(predictions3)
print(maxprob3)

In [ ]:
np.savetxt('submission2.csv',np.rec.fromarrays([test_ids,predictions3,maxprob3]),fmt=['%s','%d','%.7f'],delimiter=',',
           header='id,label,prob',comments='')

In [ ]:
probabilities4 = model4.predict(test_images_ds)
predictions4 = np.argmax(probabilities4, axis=-1)
maxprob4 = np.amax(probabilities4, axis=-1)
print(predictions4)
print(maxprob4)

In [ ]:
np.savetxt('submission3.csv',np.rec.fromarrays([test_ids,predictions4,maxprob4]),fmt=['%s','%d','%.7f'],delimiter=',',
           header='id,label,prob',comments='')

In [ ]:

probabilities5 = model5.predict(test_images_ds)
predictions5 = np.argmax(probabilities5, axis=-1)
maxprob5 = np.amax(probabilities5, axis=-1)
print(predictions5)
print(maxprob5)


In [ ]:

np.savetxt('submission4.csv',np.rec.fromarrays([test_ids,predictions5,maxprob5]),fmt=['%s','%d','%.7f'],delimiter=',',
           header='id,label,prob',comments='')


In [ ]:
"""
# Ensembling Option1
preddf1 = pd.read_csv("./submission1.csv",header=0,names=['id','label1','prob1'])
preddf2 = pd.read_csv("./submission2.csv",header=0,names=['id','label2','prob2'])
preddf3 = pd.read_csv("./submission3.csv",header=0,names=['id','label3','prob3'])
preddf1.head(3)
"""

In [ ]:
# Ensembling Option2
pred1 = pd.read_csv("./submission1.csv",header=0,names=['id','label','prob'])
pred2 = pd.read_csv("./submission2.csv",header=0,names=['id','label','prob'])
pred3 = pd.read_csv("./submission3.csv",header=0,names=['id','label','prob'])
pred4 = pd.read_csv("./submission4.csv",header=0,names=['id','label','prob'])

#predx = pd.concat([pred1,pred2,pred3,pred4],ignore_index=True)
predx = pd.concat([pred1,pred2,pred3],ignore_index=True)

predy = predx.groupby(['id','label'],sort=False)['prob'].sum().reset_index()
predz = predy.loc[predy.groupby(['id'],sort=False)['prob'].idxmax()]
dfinal= predz[['id','label']]
dfinal.head(10)

In [ ]:
len(dfinal)

In [ ]:
"""
preddfx = pd.merge(preddf1,preddf2, on='id')
preddfx = pd.merge(preddfx,preddf3, on='id')
preddfx.head(3)
"""

In [ ]:
"""
dfinal = preddfx[['id']]
dfinal['label'] = 0
preddfx['mp'] = preddfx[['prob1','prob2','prob3']].max(axis=1)
preddfx['mpl'] = 0
for i in range(len(preddfx)):
    if (preddfx['prob1'][i]==preddfx['mp'][i]):
        preddfx.loc[i,'mpl'] = preddfx['label1'][i]
    elif (preddfx['prob2'][i]==preddfx['mp'][i]):
        preddfx.loc[i,'mpl'] = preddfx['label2'][i]
    elif (preddfx['prob3'][i]==preddfx['mp'][i]):
        preddfx.loc[i,'mpl'] = preddfx['label3'][i]
preddfx.tail(3)
"""

In [ ]:
"""
for j in range(len(preddfx)):
    if (preddfx['label1'][j] == preddfx['label2'][j]):
        dfinal.loc[j,'label'] = preddfx['label1'][j]
    elif (preddfx['label2'][j] == preddfx['label3'][j]):
        dfinal.loc[j,'label'] = preddfx['label2'][j]
    elif (preddfx['label3'][j] == preddfx['label1'][j]):
        dfinal.loc[j,'label'] = preddfx['label3'][j]
    else:
        dfinal['label'][j] = preddfx['mpl'][j]
"""

In [ ]:
#dfinal.tail(3)

In [ ]:
print('Generating submission.csv file...')


# Write the submission file
#np.savetxt('submission.csv',np.rec.fromarrays([test_ids, predictions2]),fmt=['%s', '%d'],delimiter=',',header='id,label',comments='')

dfinal.to_csv('submission.csv',index=False)

# Look at the first few predictions
!head submission.csv